In [1]:
import seaborn as sns
import pandas as pd
from sklearn . model_selection import train_test_split
from sklearn . preprocessing import LabelEncoder
from sklearn . linear_model import LogisticRegression
from sklearn . metrics import accuracy_score
# Load the penguins dataset
df = sns . load_dataset (" penguins ")
df. dropna ( inplace = True )
# Filter rows for 'Adelie ' and 'Chinstrap ' classes
selected_classes = ['Adelie ', 'Chinstrap ']
df_filtered = df[df['species ']. isin ( selected_classes )]. copy ()
# Make a copy to avoid the warning
# Initialize the LabelEncoder
le = LabelEncoder ()
# Encode the species column
y_encoded = le. fit_transform ( df_filtered ['species '])
df_filtered [' class_encoded '] = y_encoded
# Display the filtered and encoded DataFrame
print ( df_filtered [[ 'species ', ' class_encoded ']])
# Split the data into features (X) and target variable (y)
y = df_filtered [' class_encoded '] # Target variable
X = df_filtered . drop ([ ' class_encoded '], axis =1)

# Split the data into training and testing sets
X_train , X_test , y_train , y_test = train_test_split (X, y,
test_size =0.2 , random_state =42)
# Train the logistic regression model . Here we are using saga
# solver to learn weights .
logreg = LogisticRegression ( solver ='saga ')
logreg . fit ( X_train , y_train )
# Predict on the testing data
y_pred = logreg . predict ( X_test )
# Evaluate the model
accuracy = accuracy_score ( y_test , y_pred )
print (" Accuracy :", accuracy )
print ( logreg .coef_ , logreg . intercept_ )

ValueError: ' penguins ' is not one of the example datasets.

In [5]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the penguins dataset
df = sns.load_dataset("penguins")
df.dropna(inplace=True)

# Filter rows for 'Adelie' and 'Chinstrap' classes (exact names, no extra spaces)
selected_classes = ['Adelie', 'Chinstrap']
df_filtered = df[df['species'].isin(selected_classes)].copy()

# Initialize the LabelEncoder for the target
le = LabelEncoder()

# Encode the species column as the target (0/1)
y_encoded = le.fit_transform(df_filtered['species'])
df_filtered['class_encoded'] = y_encoded

# Display the filtered and encoded DataFrame (optional)
print(df_filtered[['species', 'class_encoded']].head())

# Build feature matrix X:
#   1) Drop the target columns ('class_encoded' and the original 'species')
#   2) One-hot encode remaining categorical features to ensure X is numeric
X = df_filtered.drop(['class_encoded', 'species'], axis=1)
X = pd.get_dummies(X, drop_first=True)

# Target
y = df_filtered['class_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train the logistic regression model with saga solver
logreg = LogisticRegression(solver='saga', max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Coefficients shape:", logreg.coef_.shape)
print("Intercept:", logreg.intercept_)


  species  class_encoded
0  Adelie              0
1  Adelie              0
2  Adelie              0
4  Adelie              0
5  Adelie              0
Accuracy: 0.6744186046511628
Coefficients shape: (1, 7)
Intercept: [-8.8967178e-05]


C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


With liblinear

In [6]:
# Train the logistic regression model with liblinear solver
logreg = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Coefficients shape:", logreg.coef_.shape)
print("Intercept:", logreg.intercept_)


Accuracy: 0.9767441860465116
Coefficients shape: (1, 7)
Intercept: [-0.03449285]


With Standard Scalar

In [ ]:
# Compare liblinear vs saga with and without StandardScaler (numeric-only scaling)

import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1) Load data and filter to binary classes
df = sns.load_dataset("penguins").dropna().copy()
df = df[df["species"].isin(["Adelie", "Chinstrap"])].copy()

# Target
y = (df["species"] == "Chinstrap").astype(int)  # Adelie -> 0, Chinstrap -> 1

# Feature set: numeric + categorical
num_cols = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]
cat_cols = ["island", "sex"]
X = df[num_cols + cat_cols].reset_index(drop=True)

# Train/test split (stratify for class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2) Preprocessing variants
preprocess_no_scale = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

preprocess_scaled = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

# 3) Models
liblinear = LogisticRegression(solver="liblinear", max_iter=1000, random_state=42)
saga = LogisticRegression(solver="saga", penalty="l2", max_iter=1000, random_state=42)

# 4) Pipelines
pipelines = {
    "liblinear_no_scale": Pipeline([("prep", preprocess_no_scale), ("clf", liblinear)]),
    "liblinear_scaled":   Pipeline([("prep", preprocess_scaled),   ("clf", liblinear)]),
    "saga_no_scale":      Pipeline([("prep", preprocess_no_scale), ("clf", saga)]),
    "saga_scaled":        Pipeline([("prep", preprocess_scaled),   ("clf", saga)]),
}

# 5) Fit, predict, evaluate
results = {}
for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc

# 6) Print results
print("Accuracy comparison (same split):")
for k, v in results.items():
    print(f"{k:>20s}: {v:.4f}")


Accuracy comparison (same split):
  liblinear_no_scale: 0.9767
    liblinear_scaled: 1.0000
       saga_no_scale: 0.6744
         saga_scaled: 1.0000


C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
